In [ ]:
##This is the diver for the generic bootstrap particle filter

from pylab import *
from scipy.integrate import odeint
import ipdb

In [ ]:
# %load L63_vectorized.py
def L63_vectorized(state,t):
    
    """This is the vectorized derivative for the Lorenz 63 model"""
    # Define the system parameters
    sigma = 10.0
    rho   = 28.0
    beta  = 8.0/3.0
    
    # Reshape the state vector to apply the derivative  
    particles = len(state)/3
    state = reshape(state,[particles,3])
    
    # unpack the state variables
    X = state[:,0]
    Y = state[:,1]
    Z = state[:,2]

    dx = sigma*(Y-X)
    dy = X*(rho - Z) - Y
    dz = X*Y - beta*Z
    

    
    deriv = array([dx,dy,dz]).transpose()
    deriv = reshape(deriv,particles*3)
    
    return deriv

In [ ]:
# %load particle_filter.py
def  bootstrap(model,initial,ens_size,weights,interval,nanl,tanl,obs,Q):

    """This is a general bootstrap particle filter function
    
    The fields to supply are the derivative function `model', the initial cloud
    'initial', the model dimension and cloud size,
    the particle weights at initialization `weights', the interval 
    to integrate the cloud '`interval', the observation with which to 
    update the cloud at the end analysis time, and the inverse of the 
    observational error covaraince `Q'.
    
    We assume that the model is vectorized to accept a cloud of initial
    conditions of the shape [model_dimension, cloud_size]."""
    ipdb.set_trace()
    
    # store the analysis times indices in the full integration interval
    a_time = array(range(0,len(interval)+1,tanl))-1
    a_time[0] = 0
    
    # storage matrix for the trajectories
    traj = zeros([len(interval),intial.size])
    
    for i in range(nanl):
        # integrate the initial cloud to the analysis time
        traj[a_time[i]:a_time[i+1],:] = odeint(model,initial,
                                               interval[a_time[i]:a_time[i+1]])
        
        # reset intial condition
        intial = traj[a_time[i+1]]
        
        #compute the likelyhood function
        innov = reshape(initial,[ens_size,model_dim])
        """need to check if the dimensions here are all right"""
        innov = obs - innov.transpose()
        temp  = sum(Q.dot(innov)*innov,axis=0)
        likelyhood = exp(-0.5*temp)**(1.0/3.0)
    
        weights = weights*likelyhood
    
    return [traj,weights]

In [ ]:
## System Parameters

# Define the number of particles in the ensemble
root = 2
particle_number = root**3

# Spin time
spin_end = 10

# Time step
dt = .01

# Spin interval
spin = linspace(0,spin_end,spin_end/dt)

# Obs Err variance (% of climate variance) 
obs_var = 0.01

# Analysis interval
tanl = 50

# Number of Analyses
nanl = 100

# Experiment length
exp_len = tanl*nanl
exp_int = linspace(0,exp_len*dt,exp_len)

# state dimension
state_dim = 3

In [ ]:
## Spin up the ensemble of initial conditions for the filter and `truth'

# Assign cubes of initial conditions for the ensemble spin up, run the spin,calculate spin mean, 
# and variance to set initial conditions for the runs
ens_base = randint(-100,100)

# Define cube of initial conditions for the spin up 
cube = zeros([particle_number,state_dim])

for i in range(root):
    for j in range(root):
        for k in range(root):            
            cube[(root**2)*i + root*j + k, :] =(array([k*exp(-1),j*exp(-1),i*exp(-1)])+ens_base)
                                                      

# Spin up the initial cloud
spin_cloud = reshape(cube,3*particle_number)
spin_cloud = odeint(L63_vectorized,spin_cloud,spin)
spin_cloud = reshape(spin_cloud,[len(spin),particle_number,state_dim])
spun_cloud = spin_cloud[-1,:,:]

In [ ]:
## Calcualte the environmental statistics

#Determine the mean for the spin cloud at each time step
spin_mean = mean(spun_cloud,axis=0)

#Calculate variance of the mean along spin up
cl_var = 2*var(spun_cloud,axis=0)

#Observational Error covariance stored    
R = cl_var*eye(state_dim)*obs_var
Q = inv(R)

In [ ]:
## Create the `perfect' model experiment

# Generate random ensemble member to be initializaiton for `truth'
P = randint(0,particle_number)
truth = spun_cloud[P,:]

# `Truth' state is eliminated from the particle cloud
PF_cloud = delete(spun_cloud,P,0)

# Propagate a `true' trajectory to generate the observations
truth = odeint(L63_vectorized,truth,exp_int)
truth = reshape(truth,[exp_len,3])

#Create observations with error, note the tanl-1 so that accounts for index beginning at 0
obs = truth[tanl-1:exp_len:tanl,:] + (randn(nanl,3)*sqrt(obs_var*cl_var))

In [ ]:
## Run bootstrap filter on the sequence of observations with the intialized cloud

# Set the weights and initial conditions
ens_size = len(PF_cloud[:,0])
weights = 1.0/(ens_size)
weights = weights*ones(ens_size)

PF_cloud = reshape(PF_cloud,PF_cloud.size)

%debug

[traj,weights] = bootstrap(L63_vectorized,PF_cloud,ens_size,weights,exp_int,nanl,tanl,obs,Q)
